[Source](https://www.tensorflow.org/tutorials/text/nmt_with_attention)



# Chargez les librairies

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd

from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
try:
  !pip install sacrebleu
except:
  import sacrebleu

     |████████████████████████████████| 61kB 2.8MB/s 


# Chargez et préparez le jeu de données

Après avoir charger l'ensemble des données, voici les étapes que nous allons suivre pour préparer les données:

 - Ajoutez un jeton de début et de fin à chaque phrase.
 - Nettoyez les phrases en supprimant les caractères spéciaux.
 - Créez un index de mots et un index de mots inversé (mappage de dictionnaires de mot → id et id → mot).
 - Remplissez chaque phrase à une longueur maximale.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
dossier = '/gdrive/MyDrive/Zindi/AI4D-TakwimuLab-Machine-Translation-Challenge/'

In [ ]:
train = pd.read_csv(dossier + '/Data/Train.csv')
test = pd.read_csv(dossier + '/Data/Test.csv')
sub = pd.read_csv(dossier + '/Data/SampleSubmission.csv')

In [ ]:
train.head()

,ID,French,Target_Language,Target
0,ID_AADNDxdl,Mon père,Fon,Tɔ ce
1,ID_AAFQhmDr,Mettez-vous en rang.,Fon,Mi tò miɖéé
2,ID_AAHVDMdq,"Sénégal, Côte d'Ivoire, Guinée, Ghana, on déco...",Ewe,"Sénégal, Côte d'Ivoire, Guinée, Ghana, siwo ƒe..."
3,ID_AAJfVHEH,Son doigt lui fait mal.,Fon,Alɔvi tɔn ɖo vivɛ wɛ
4,ID_AAOJuhzN,La pluie a commencé.,Fon,Jì bɛ́


In [ ]:
test.head()

,ID,French,Target_Language
0,ID_AAAAhgRX,Très fière d’elle,Ewe
1,ID_AAGuzGzi,Tous ces grands artistes viendront au Benin po...,Fon
2,ID_AAuiTPkQ,Ce programme va travailler à améliorer les con...,Fon
3,ID_ACYgGXTq,Quels sont les questions récurrentes de ceux ...,Fon
4,ID_AChdWHyF,Grosse bagnolle,Ewe


In [ ]:
sub.head()

,ID,Target
0,ID_nNUszvhL,0
1,ID_HbooJGwk,0
2,ID_WMxavljL,0
3,ID_pldWaUHs,0
4,ID_gjCwdavy,0


In [ ]:
len(test), len(sub)

(5893, 5893)

In [ ]:
len(train)

75487

In [ ]:
fon_train = train[train.Target_Language == "Fon"].reset_index(drop=True)
ewe_train = train[train.Target_Language == "Ewe"].reset_index(drop=True)

In [ ]:
fon_train

,ID,French,Target_Language,Target
0,ID_AADNDxdl,Mon père,Fon,Tɔ ce
1,ID_AAFQhmDr,Mettez-vous en rang.,Fon,Mi tò miɖéé
2,ID_AAJfVHEH,Son doigt lui fait mal.,Fon,Alɔvi tɔn ɖo vivɛ wɛ
3,ID_AAOJuhzN,La pluie a commencé.,Fon,Jì bɛ́
4,ID_AAOZhyDe,Les garçons et les filles sont au cours élémen...,Fon,Nyɔnu lε kpo sunnu lε kpo ɖo wemaxomε ayi jε t...
...,...,...,...,...
53129,ID_zzsOfbtD,Reconduis-le.,Fon,Kpl’ɛ́ yì
53130,ID_zztJChgi,Le vent pousse le nuage,Fon,Jɔhɔn ɔ ɖo akpɔkpɔ sisε wε.
53131,ID_zztqxmJe,"Si mon mari conduit, c’est que ce n’est pas lo...",Fon,"Nu asu ce kun hun ɔ, hun e lin a nε"
53132,ID_zzxVgzgw,"Cette année, je vais épouser celle-là.",Fon,"Xwe elɔ mɛ, un na wli alɔ x’ɛ"


In [ ]:
fon_test = test[test.Target_Language == "Fon"].reset_index(drop=True)
ewe_test = test[test.Target_Language == "Ewe"].reset_index(drop=True)

In [ ]:
len(fon_train), len(ewe_train)

(53134, 22353)

In [ ]:
len(fon_test), len(ewe_test)

(2929, 2964)

In [ ]:
# Converts the unicode file to ascii
#def unicode_to_ascii(s):
#  return ''.join(c for c in unicodedata.normalize('NFD', s)
#      if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  #w = unicode_to_ascii(w.lower().strip())
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
preprocess_sentence("Ɖo gan tantɔn kpo cεju afɔton")

'<start> ɖo gan tantɔn kpo cεju afɔton <end>'

In [ ]:
def create_dataset(df, langue):

  filtered_df = df[df.Target_Language == langue].copy()
  filtered_df['French'] = filtered_df['French'].apply(preprocess_sentence)
  filtered_df['Target'] = filtered_df['Target'].apply(preprocess_sentence)

  fr = list(filtered_df['French'])
  targ = list(filtered_df['Target'])

  return targ, fr

In [ ]:
fon, fr = create_dataset(train, "Fon")
print(fr[6])
print(fon[6])

<start> il l’a fait , bien sûr . <end>
<start> e bló hŭn <end>


In [ ]:
ewe, fr = create_dataset(train, "Ewe")
print(fr[5])
print(ewe[5])

<start> on essaie donc de deviner ce qui se passe à  partir des rares mots lâchés en français , et on en rigole comme des folles , <end>
<start> miedze agbagba susu nusi gblɔm wole xe koanu ɖe frãsɛ ɖekaɖeka siwo miese la dzi <end>


In [ ]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='', oov_token = 'OOV')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [ ]:
def load_dataset(df, langue):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(df, langue)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [ ]:
train.shape

(75487, 4)

In [ ]:
# Try experimenting with the size of that dataset

train_exp = train.sample(frac=.5, random_state=92).copy()
train_exp.Target_Language.value_counts()

Fon    26630
Ewe    11114
Name: Target_Language, dtype: int64

In [ ]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(train_exp, 'Fon')

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [ ]:
max_length_inp, max_length_targ

(56, 71)

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

21304 21304 5326 5326


In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
52 ----> cela
21 ----> ne
6334 ----> m’intéresse
16 ----> pas
4 ----> .
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
8 ----> un
47 ----> ma
129 ----> kan
277 ----> byɔ́
3 ----> <end>


# Créer un ensemble de données tf.data

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 56]), TensorShape([64, 71]))

# Ecrire le modèle d'encodeur et de décodeur

Le diagramme suivant montre que chaque mot d'entrée se voit attribuer un poids par le mécanisme d'attention qui est ensuite utilisé par le décodeur pour prédire le mot suivant dans la phrase. L'image et les formules ci-dessous sont un exemple de mécanisme d'attention tiré de l'article de [Luong](https://arxiv.org/abs/1508.04025v5).

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

L'entrée est soumise à un modèle d'encodeur qui nous donne la sortie d'encodeur de forme (batch_size, max_length, hidden_size) et l'état de forme caché de l'encodeur (batch_size, hidden_size) .

Voici les équations qui sont implémentées:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

L' attention de Bahdanau est utilisé pour l'encodeur.

La notation suivante est utilisé afin d'écrire la forme simplifiée:

* FC = couche entièrement connectée (dense)
* EO = sortie codeur
* H = état caché
* X = entrée du décodeur

Et le pseudo-code:

* ```score = FC(tanh(FC(EO) + FC(H)))```
* ```attention weights = softmax(score, axis = 1)``` . Softmax par défaut est appliqué sur le dernier axe mais ici nous voulons l'appliquer sur le 1er axe , puisque la forme du score est (batch_size, max_length, hidden_size) . 
* ```Max_length``` est la longueur de notre entrée. Puisque nous essayons d'attribuer un poids à chaque entrée, softmax doit être appliqué sur cet axe.
* ```context vector = sum(attention weights * EO, axis = 1)``` . Même raison que ci-dessus pour choisir l'axe comme 1.
*  ```embedding output``` = L'entrée du décodeur X passe à travers une couche d'incorporation.
* ```merged vector = concat(embedding output, context vector)```
* Ce vecteur fusionné est ensuite donné au GRU

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 56, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 56, 1)


In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 9749)


# Définir l'optimiseur et la fonction de perte

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

# Points de contrôle (enregistrement basé sur les objets)

In [ ]:
checkpoint_dir = dossier + 'training_checkpoints_fon' #for fon
#checkpoint_dir = dossier + 'training_checkpoints_ewe' #for ewe

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

# L'entraînement

1-  Passez l' entrée à travers le codeur qui renvoie la sortie du codeur et l' état caché du codeur .

2-  La sortie du codeur, l'état caché du codeur et l'entrée du décodeur (qui est le jeton de démarrage ) sont transmis au décodeur.

3-  Le décodeur renvoie les prédictions et l' état caché du décodeur .

4-  L'état caché du décodeur est ensuite renvoyé dans le modèle et les prédictions sont utilisées pour calculer la perte.

5-  Utilisez le forçage de l'enseignant pour décider de la prochaine entrée du décodeur.

6-  Le forçage par l'enseignant est la technique dans laquelle le mot cible est transmis comme entrée suivante au décodeur.

7-  La dernière étape consiste à calculer les dégradés et à l'appliquer à l'optimiseur et à la rétropropagation.


In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.9318
Epoch 1 Batch 100 Loss 0.6469
Epoch 1 Batch 200 Loss 0.5606
Epoch 1 Batch 300 Loss 0.5354
Epoch 1 Loss 0.5612
Time taken for 1 epoch 350.70720314979553 sec

Epoch 2 Batch 0 Loss 0.4639
Epoch 2 Batch 100 Loss 0.4700
Epoch 2 Batch 200 Loss 0.4544
Epoch 2 Batch 300 Loss 0.4054
Epoch 2 Loss 0.4532
Time taken for 1 epoch 285.4874322414398 sec

Epoch 3 Batch 0 Loss 0.4431
Epoch 3 Batch 100 Loss 0.4214
Epoch 3 Batch 200 Loss 0.3491
Epoch 3 Batch 300 Loss 0.3578
Epoch 3 Loss 0.3882
Time taken for 1 epoch 283.85774755477905 sec

Epoch 4 Batch 0 Loss 0.3312
Epoch 4 Batch 100 Loss 0.2794
Epoch 4 Batch 200 Loss 0.3071
Epoch 4 Batch 300 Loss 0.3449
Epoch 4 Loss 0.3279
Time taken for 1 epoch 284.659295797348 sec

Epoch 5 Batch 0 Loss 0.2800
Epoch 5 Batch 100 Loss 0.2798
Epoch 5 Batch 200 Loss 0.2315
Epoch 5 Batch 300 Loss 0.2551
Epoch 5 Loss 0.2677
Time taken for 1 epoch 283.7257797718048 sec

Epoch 6 Batch 0 Loss 0.2176
Epoch 6 Batch 100 Loss 0.1764
Epoch 6 Batch 200 Los

# Traduire

* La fonction d'évaluation est similaire à la boucle d'apprentissage, sauf que 
nous n'utilisons pas le forçage de l'enseignant ici. L'entrée du décodeur à chaque pas de temps correspond à ses prédictions précédentes avec l'état caché et la sortie du codeur.
* Arrêtez de prédire quand le modèle prédit le jeton de fin .
* Et stockez les poids d'attention pour chaque pas de temps .

In [ ]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index.get(i, 1) for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  #print('Input: %s' % (sentence))
  #print('Predicted translation: {}'.format(result))

  #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  #plot_attention(attention_plot, sentence.split(' '), result.split(' '))
  result = result.replace('<end>', '')
  return result.strip()

# Restaurez le dernier point de contrôle et testez

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
# checkpoint_dir = dossier + 'training_checkpoints_ewe' #for ewe

checkpoint_dir = dossier + 'training_checkpoints_fon' #for fon

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
pd.set_option('display.max_colwidth', 100000 )

In [ ]:
fon_train
#ewe_train

,ID,French,Target_Language,Target
0,ID_AADNDxdl,Mon père,Fon,Tɔ ce
1,ID_AAFQhmDr,Mettez-vous en rang.,Fon,Mi tò miɖéé
2,ID_AAJfVHEH,Son doigt lui fait mal.,Fon,Alɔvi tɔn ɖo vivɛ wɛ
3,ID_AAOJuhzN,La pluie a commencé.,Fon,Jì bɛ́
4,ID_AAOZhyDe,Les garçons et les filles sont au cours élémentaire .,Fon,Nyɔnu lε kpo sunnu lε kpo ɖo wemaxomε ayi jε tɔn mε
...,...,...,...,...
53129,ID_zzsOfbtD,Reconduis-le.,Fon,Kpl’ɛ́ yì
53130,ID_zztJChgi,Le vent pousse le nuage,Fon,Jɔhɔn ɔ ɖo akpɔkpɔ sisε wε.
53131,ID_zztqxmJe,"Si mon mari conduit, c’est que ce n’est pas loin .",Fon,"Nu asu ce kun hun ɔ, hun e lin a nε"
53132,ID_zzxVgzgw,"Cette année, je vais épouser celle-là.",Fon,"Xwe elɔ mɛ, un na wli alɔ x’ɛ"


In [ ]:
#test[test.Target_Language=='Fon'].sample(5)
fon_train.sample(5)
#ewe_train.sample(5)

,ID,French,Target_Language,Target
29353,ID_cZmdJiEI,très beau (denrée),Fon,nylă kàn
14188,ID_NrOPyicY,Ce monsieur est le chef de notre localité.,Fon,Mɛɖaxó élɔ́ ɔ́ kpà'cɛ̀ dó tò mìtɔ́n mɛ̀
7022,ID_Gqhlsgcw,A ta santé !,Fon,Sin n’í yí kɔ̀ !
44021,ID_qzXsCxId,La poule pond des oeufs .,Fon,Koklo ɔ ɖo azin ɖo wε
31913,ID_fAuzKJrH,Mais ta mère me rend fou.,Fon,Nɔ ce nɔ do taɖu mi


In [ ]:
translate("Je vais au marche")

'un ɖo xu to mi'

In [ ]:
print(fon_train['French'][12])
#print(ewe_train['French'][10])
#translate(fon_test['French'][12])

C'est très bien.


In [ ]:
test_bleu = train_exp[train_exp.Target_Language=='Fon'].copy()

#test_bleu
#test_bleu = fon_train.copy()
#test_bleu
#test_bleu = ewe_train.copy()
#test_bleu

In [ ]:
def clean(w):
  w = w.replace('.', '').strip()
  return w

In [ ]:
# clean the dots at the end of sentences
ref = [test_bleu['Target'].apply(clean).tolist()]

In [ ]:
# clean the dots at the end of sentences
test_bleu['predictions'] = test_bleu.French.progress_apply(clean)

In [ ]:
test_bleu['predictions'] = test_bleu.predictions.progress_apply(translate)

In [ ]:
test_bleu['predictions'][:5]

60321                                                                                                                                                                       mi ɖe lɛ lo , etɛ mi nɔ blo ?
73580                                                                                                                                                                       mεsέdó - wɛndotɔ́ - wɛnsagùn
19789                                                                                                                                                                                             aza vaà
61425                                                                                                                                                                                          sìn blùblú
65617    e ɖo na ba do nu mɛ ɖi lee jlo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo e ɖɔ xo 

In [ ]:
# clean <dots> in translation
test_bleu['predictions'] = test_bleu['predictions'].progress_apply(clean)

In [ ]:
sys = test_bleu['predictions'].tolist()

In [ ]:
import sacrebleu
bleu = sacrebleu.corpus_bleu(sys, ref)
print(bleu.score,'%')

18.547239641706273 %


In [ ]:
#test.head(10).French.tolist()
test.head(10)

,ID,French,Target_Language
0,ID_AAAAhgRX,Très fière d’elle,Ewe
1,ID_AAGuzGzi,Tous ces grands artistes viendront au Benin pour prendre part à la cérémonie du lancement de l'album l'année prochaine au stade de l'Amitié.,Fon
2,ID_AAuiTPkQ,Ce programme va travailler à améliorer les conditions de vie des populations et impliquer davantage le secteur privé dans la gestion des richesses nationales.,Fon
3,ID_ACYgGXTq,"Quels sont les questions récurrentes de ceux qui sont curieux de savoir pourquoi cette initiative, notamment sur les réseaux sociaux",Fon
4,ID_AChdWHyF,Grosse bagnolle,Ewe
5,ID_AFBqjFUm,"La septième fois, penché en avant, je reçus sur moi l’eau qui avait préalablement coulé du toit de la maison",Fon
6,ID_AFFhTuyI,"Depuis lors, – de sa signature d’artiste – ne cesse de s’exprimer à travers le monde et de transmettre des messages forts au travers de ses œuvres colorées, inspirées de sa culture africaine, des personnes qu’ils croisent et de sa vision",Fon
7,ID_AGLORnKo,On a dit à Kpadonou qu'il devra repasser l'examen.,Fon
8,ID_AGZbvlsl,"Une autre personne ne me fera pas ça, donc c’est bon d’avoir d’enfant",Fon
9,ID_AGdMSyhM,Parle-nous de deux de tes œuvres « L’amour dans la vieillesse » et « le couple souriant »,Fon


In [ ]:
# clean the dots at the end of sentences
sub['predictions'] = test.French.progress_apply(clean)

In [ ]:
sub['predictions'] = sub.predictions.progress_apply(translate)

In [ ]:
# clean the dots at the end of sentences
sub['predictions'] = sub.predictions.progress_apply(clean)

# Soumission sur Zindi

In [ ]:
soumission = pd.DataFrame({'ID':sub.ID, 'Target':sub.predictions})
soumission.head()

,ID,Target
0,ID_nNUszvhL,énɛ́ ɔ́ wú zɛ̀
1,ID_HbooJGwk,mε tobútobú ɖé lɛ́ bì sín ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi ɖo ali enε ɔ mɛ bi
2,ID_WMxavljL,ali enε ɔ na xa togun enɛ ɔ kpɔn bɔ e na xa togun towe ɖɔhun
3,ID_pldWaUHs,"mɛɖe ɖe lɛ mɔ ye ma ɖu ɖo ye ɖɔ ye ɖo to lɛ ɔ , ɖe lɛ mɔ ye ma ɖu ɖo ye ɖɔ ye ɖo to lɛ ɔ , ɖe lɛ mɔ ye ma ɖu ɖo ye ɖɔ ye ɖo to lɛ ɔ , ɖe lɛ mɔ ye ma ɖu ɖo ye ɖɔ ye ɖo to lɛ ɔ , ɖe lɛ mɔ ye ma ɖu ɖo ye ɖɔ ye"
4,ID_gjCwdavy,degenɔ kpikpò - degénɔ́kpikpó - degénɔ́kpúkpó - degénɔ́kpúkpwé


In [ ]:
soumission.to_csv(dossier+'/Soumissions/soumission2.csv', index=False)

FIN